In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
from PIL import Image
import PIL.ImageOps

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.utils
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import os

import shutil
import torchvision.transforms as T
import csv
try:
    from PIL import Image
except ImportError:
    import Image

# from googletrans import Translator, constants
# from pprint import pprint
# import pytesseract

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

%cd /content/drive/My Drive/phishing1


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/phishing1


In [ ]:
# #OCR
# !pip3 install googletrans
# !pip3 install googletrans==3.1.0a0
# !sudo apt install tesseract-ocr
# !pip install pytesseract

In [ ]:
# #create the Siamese Neural Network Datahacker mit Layeranpassung
# class SiameseNetwork(nn.Module):

#     def __init__(self):
#         super(SiameseNetwork, self).__init__()

#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(3, 96, kernel_size=11,stride=4),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(3, stride=2),

#             nn.Conv2d(96, 256, kernel_size=5, stride=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),

#             nn.Conv2d(256, 384, kernel_size=3,stride=1),
#             nn.ReLU(inplace=True)
#         )

#         # Setting up the Fully Connected Layers
#         self.fc1 = nn.Sequential(
#             nn.Linear(31104, 9096),
#             nn.ReLU(inplace=True),

#             nn.Linear(9096, 1024),
#             nn.ReLU(inplace=True),

#             nn.Linear(1024, 256),
#             nn.ReLU(inplace=True),

#             nn.Linear(256,2)


#         )




#     def forward_once(self, x):
#         # This function will be called for both images
#         # It's output is used to determine the similiarity
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         # In this function we pass in both images and obtain both vectors
#         # which are returned
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)

#         return output1, output2

In [ ]:
# #AlexNet
# #https://gist.github.com/bigsnarfdude/f90b7a3bc6965ffc4c5e89e1d6a6c7b0
# #create the Siamese Neural Network
# class SiameseNetwork(nn.Module):

#     def __init__(self):
#         super(SiameseNetwork, self).__init__()

#         # Setting up the Sequential of CNN Layers
#         self.cnn1 = nn.Sequential(
#             nn.Conv2d(3, 64, kernel_size=10, stride=2, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#             nn.Conv2d(64, 192, kernel_size=5, padding=2),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#             nn.Conv2d(192, 384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(384, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.Conv2d(256, 256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
#             nn.MaxPool2d(2, stride=2),
#         )


#         # Setting up the Fully Connected Layers
#         self.fc1 = nn.Sequential(


#           #(1x9216 and 12544x512)
#           #Ram durch viel zu groß

#              nn.Linear(43264 , 512),
#              nn.ReLU(inplace=True),
#              nn.Linear(512, 256),
#              nn.ReLU(inplace=True),
#              nn.Linear(256, 2)
#         )

#     def forward_once(self, x):
#         # This function will be called for both images
#         # It's output is used to determine the similiarity
#         output = self.cnn1(x)
#         output = output.view(output.size()[0], -1)
#         output = self.fc1(output)
#         return output

#     def forward(self, input1, input2):
#         # In this function we pass in both images and obtain both vectors
#         # which are returned
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)

#         return output1, output2

In [ ]:
#60epoch_adam0.00010_batch16_Datahackertrusted_list_1img
#60epoch_adam0.00010_batch16_Alexnettrusted_list_1img

state_dict = torch.load('/content/drive/My Drive/.pth')
print(state_dict.keys())
model = SiameseNetwork().cuda()

odict_keys(['cnn1.0.weight', 'cnn1.0.bias', 'cnn1.3.weight', 'cnn1.3.bias', 'cnn1.6.weight', 'cnn1.6.bias', 'cnn1.9.weight', 'cnn1.9.bias', 'fc1.0.weight', 'fc1.0.bias', 'fc1.2.weight', 'fc1.2.bias', 'fc1.4.weight', 'fc1.4.bias'])


In [ ]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# class SiameseNetworkDataset(Dataset):
#     def __init__(self,imageFolderDataset,transform=None):
#         self.imageFolderDataset = imageFolderDataset
#         self.transform = transform

#     def __getitem__(self,index):
#         img0_tuple = random.choice(self.imageFolderDataset.imgs)

#         #We need to approximately 50% of images to be in the same class
#         should_get_same_class = random.randint(0,1)
#         if should_get_same_class:
#             while True:
#                 #Look untill the same class image is found
#                 img1_tuple = random.choice(self.imageFolderDataset.imgs)
#                 if img0_tuple[1] == img1_tuple[1]:
#                     break
#         else:

#             while True:
#                 #Look untill a different class image is found
#                 img1_tuple = random.choice(self.imageFolderDataset.imgs)
#                 if img0_tuple[1] != img1_tuple[1]:
#                     break

#         img0 = Image.open(img0_tuple[0])
#         img1 = Image.open(img1_tuple[0])
# #ausprobieren mit in grau umwandeln und dann in rgb
#         img0 = img0.convert('RGB')
#         img1 = img1.convert('RGB')

#         if self.transform is not None:
#             img0 = self.transform(img0)
#             img1 = self.transform(img1)

#         return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

#     def __len__(self):
#         return len(self.imageFolderDataset.imgs)

In [ ]:
class SiameseNetworkDataset(Dataset):
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset
        self.transform = transform

    def __getitem__(self,index):
        img0_tuple = self.imageFolderDataset.imgs[index]
        # print(img0_tuple, "img tuple", "\n")
        # print(index, "INDEX", "\n")
        #Choose a random index for the second image
        img1_index = random.randint(0, len(self.imageFolderDataset) - 1)
        #print(img1_index, "img 1 INDEX", "\n")
        #Ensure that the two images are not the same
        while img1_index == index:
            img1_index = random.randint(0, len(self.imageFolderDataset) - 1)

        img1_tuple = self.imageFolderDataset.imgs[img1_index]

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])

        img0 = img0.convert('RGB')
        img1 = img1.convert('RGB')

        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)

        return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [ ]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# class SiameseNetworkDataset(Dataset):
#     def __init__(self,imageFolderDataset,transform=None):
#         self.imageFolderDataset = imageFolderDataset
#         self.transform = transform

#     def __getitem__(self,index):
#         img0_tuple = self.imageFolderDataset.imgs[index]

#         #Choose a random index for the second image
#         img1_index = random.randint(0, len(self.imageFolderDataset) - 1)

#         #Ensure that the two images are not the same
#         while img1_index == index:
#             img1_index = random.randint(0, len(self.imageFolderDataset) - 1)

#         img1_tuple = self.imageFolderDataset.imgs[img1_index]

#         img0 = Image.open(img0_tuple[0])
#         img1 = Image.open(img1_tuple[0])

#         img0 = img0.convert('RGB')
#         img1 = img1.convert('RGB')

#         if self.transform is not None:
#             img0 = self.transform(img0)
#             img1 = self.transform(img1)

#         return img0, img1, torch.from_numpy(np.array([int(img1_tuple[1] != img0_tuple[1])], dtype=np.float32))

#     def __len__(self):
#         return len(self.imageFolderDataset.imgs)

In [ ]:
folder_dataset = datasets.ImageFolder(root="/content/drive/My Drive/.../")

In [ ]:
# Resize the images and transform to tensors
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
transformation = transforms.Compose([transforms.Resize((224,224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                                    ])

            # tv.transforms.Resize((224, 224)),
            # tv.transforms.RandomCrop((224, 224)),
            # tv.transforms.RandomHorizontalFlip(),
            # tv.transforms.ToTensor(),
            # tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),

# Initialize the network
#siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                       # transform=transformation)

In [ ]:
# Creating some helper functions
def imshow(img, text=None):
    npimg = img.numpy()
    plt.axis("off")
    #plt.figure(figsize=(40,40))
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})

    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.show()

In [ ]:
# Locate the test dataset and load it into the SiameseNetworkDataset
folder_dataset_test = datasets.ImageFolder(root="/content/drive/My Drive/.../")
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transformation)
test_dataloader = DataLoader(siamese_dataset, num_workers=2, batch_size=1, shuffle=False)


# Locate the test dataset and load it into the SiameseNetworkDataset
folder_dataset_test_original = datasets.ImageFolder(root="/content/drive/My Drive/.../")
siamese_dataset_original = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test_original,
                                        transform=transformation)
test_dataloader_original = DataLoader(siamese_dataset_original, num_workers=2, batch_size=1, shuffle=False)





print(folder_dataset_test)
print(folder_dataset_test_original)
#


# Grab one image that we are going to test
dataiter = iter(test_dataloader)
dataiter_ori = iter(test_dataloader_original)
file = open('60epoch_adam0.00010_batch16_Alexnet_Alibaba.txt', 'w')


for trusted1, _, _ in dataiter_ori:
    print(1)

    for x1, _, _ in test_dataloader:

      concatenated = torch.cat((trusted1, x1), 0)



      output1, output2 = model(trusted1.cuda(), x1.cuda())

      euclidean_distance = F.pairwise_distance(output1, output2)

      distance = f' {euclidean_distance.item():.2f}'


      #img = transtoimg(x1)




      #image = img.convert('RGB')
      #extracted_information = pytesseract.image_to_string(image)
      #print(extracted_information)
      #file.write(str(distance) + '\n')
      file.write(str(distance).replace('.', ',') + '\n')
      imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')

file.close()

In [ ]:
# # Locate the test dataset and load it into the SiameseNetworkDataset
# folder_dataset_test = datasets.ImageFolder(root="/content/drive/My Drive/phishing1/nontrusted_list_10classes/")
# siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
#                                         transform=transformation)
# test_dataloader = DataLoader(siamese_dataset, num_workers=2, batch_size=1, shuffle=False)


# # Locate the test dataset and load it into the SiameseNetworkDataset
# folder_dataset_test_original = datasets.ImageFolder(root="/content/drive/My Drive/phishing1/eval_alibaba/")
# siamese_dataset_original = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test_original,
#                                         transform=transformation)
# test_dataloader_original = DataLoader(siamese_dataset_original, num_workers=2, batch_size=1, shuffle=False)


# # Grab one image that we are going to test
# dataiter = iter(test_dataloader)
# dataiter_ori = iter(test_dataloader_original)
# file = open('60epoch_adam0.00010_batch16_Alexnet_Alibaba.txt', 'w')



# for trusted1, _, _ in dataiter_ori:
#     for x1, _, _ in test_dataloader:

#       concatenated = torch.cat((trusted1, x1), 0)

#       output1, output2 = model(trusted1.cuda(), x1.cuda())

#       euclidean_distance = F.pairwise_distance(output1, output2)
#       distance = f' {euclidean_distance.item():.2f}'
#       #file.write(str(distance) + '\n')
#       file.write(str(distance).replace('.', ',') + '\n')
#       imshow(torchvision.utils.make_grid(concatenated), f'Dissimilarity: {euclidean_distance.item():.2f}')

# file.close()


In [ ]:
# folder_dataset = datasets.ImageFolder(root="/content/drive/My Drive/phishing1/nontrusted_list_10classes/")
# #OCR Tesseract
# translator = Translator()
# txtfile = 'ocr_nontrusted_list_10classes.txt'
# file = open(txtfile, 'w')
# file.write("email;password;sign in;brand"+ '\n')

# for img, _ in folder_dataset:

#     # Verwendung des Bildes für OCR
#     extracted_information = pytesseract.image_to_string(img)


#     translation = translator.translate(extracted_information)
#     print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")
#     print("Hier src" +translation.src)
#     text = translation.text
#     print("Hier text" + text + "\n")
#     file.write(str((int("email".lower() in text.lower()or "username".lower() in text.lower() or "User ID".lower() in text.lower() or "account name".lower() in text.lower()))) + ";")
#     file.write(str(int("password".lower() in text.lower()))+ ";")
#     file.write(str(int("sign in".lower() in text.lower() or "log in".lower() in text.lower()))+ ";")



#     brand = ['alibaba', 'amazon', 'American Express', 'apple', 'chase', 'facebook', 'instagram', 'linkedin', 'paypal', 'steam']
#     found_brands = [keyword for keyword in brand if keyword.lower() in text.lower()]

#     if found_brands:

#         for brand in found_brands:
#             file.write(str(brand)+ '\n')
#     else:
#         file.write("no brand"+ '\n')

# file.close()

In [ ]:
# text = textall
# txtfile = 'ocr_10classes.txt'
# file = open(txtfile, 'w')
# file.write("email;password;sign in;brand"+ '\n')



# print(text)

# file.write(str((int("email" in text.lower() or "account name" in text.lower()))) + ";")
# file.write(str(int("password" in text.lower()))+ ";")
# file.write(str(int("sign in" in text.lower() or "log in" in text.lower()))+ ";")


# brand = ['alibaba', 'amazon', 'amex', 'apple', 'chase', 'facebook', 'instagram', 'linkedin', 'paypal', 'steam']
# found_brands = [keyword for keyword in brand if keyword.lower() in text.lower()]

# if found_brands:

#     for brand in found_brands:
#         file.write(str(brand))
# else:
#     file.write("no brand"+ '\n')

# file.close()

In [ ]:
# !pip3 install googletrans==3.1.0a0